<a href="https://colab.research.google.com/github/visiont3lab/haar-cascade-custom-training/blob/main/haar_cascade_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Collect all cleaned data

In [4]:
!git clone https://github.com/visiont3lab/haar-cascade-custom-training.git

Cloning into 'haar-cascade-custom-training'...
remote: Enumerating objects: 3096, done.
remote: Counting objects: 100% (3096/3096), done.
remote: Compressing objects: 100% (3068/3068), done.
remote: Total 3096 (delta 25), reused 3091 (delta 23), pack-reused 0
Receiving objects: 100% (3096/3096), 234.71 MiB | 18.79 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Checking out files: 100% (3399/3399), done.


## Create haar cascade sript for training

In [12]:
%cd /content/haar-cascade-custom-training/object-detector

/content/haar-cascade-custom-training/object-detector


In [13]:
%%writefile train_haar.sh
#!/bin/bash

# Info
# https://answers.opencv.org/question/4368/traincascade-error-bad-argument-can-not-get-new-positive-sample-the-most-possible-reason-is-insufficient-count-of-samples-in-given-vec-file/
# https://docs.opencv.org/master/dc/d88/tutorial_traincascade.html
# https://dbloisi.github.io/tutorial/balldetection.html
# To speed up training recompile opencv multitrheading (TBB ON)
# Remove installed opencv sudo rm /usr/local/{bin,lib}/*opencv* 
# https://answers.opencv.org/question/10/how-to-build-opencv-with-tbb-support/
# cd /home/manuel/thermohuman_ws/lib/opencv-3.4.0/release && cmake -D CMAKE_BUILD_TYPE=RELEASE -D WITH_CUDA=OF -D CMAKE_INSTALL_PREFIX=/usr/local -D WITH_TBB=ON -D BUILD_PYTHON_SUPPORT=ON  ..

# Set input
results="/content/haar-cascade-custom-training/object-detector/cascade_training/results"
images_folder="/content/haar-cascade-custom-training/object-detector/dataset/mask-resize"
images_txt="/content/haar-cascade-custom-training/object-detector/training.txt"           #337
backgrounds_txt="/content/haar-cascade-custom-training/object-detector/background.txt" #1109
num_images=600 #441  # 4863 not ok
num_pos=400   #400 # 500 # almost 90% to select this number https://answers.opencv.org/question/24241/my-classifier-using-haar-cascade-can-not-detect-anything/
num_neg=800 # 250

# remove results and trainining txt files
if [ -d $results ]; then rm -Rf $results; fi
if [ -f positive.vec ]; then rm -f positive.vec; fi

# create resutl folder
mkdir -p $results

# Create positive.vec
# opencv_traincascade -data results -vec positive.vec -bg bg.txt -numPos 500 -numNeg 250 -numStages 10 -w 24 -h 24 
opencv_createsamples -info $images_txt -vec positive.vec -num $num_images   -w 24 -h 24


# Train
#opencv_traincascade -data $results -vec positive.vec -bg $backgrounds_txt -numPos $num_pos -numNeg $num_neg # default: -minHitRate 0.995  -maxFalseAlarmRate 0.5 
opencv_traincascade -data $results -vec positive.vec -bg $backgrounds_txt -numStages 20 -numPos $num_pos -numNeg  $num_neg  -minHitRate 0.995  -maxFalseAlarmRate 0.5 -w 24 -h 24 # -mode ALL  #-precalcValBufSize 4096 -precalcIdxBufSize 4096


Overwriting train_haar.sh


## Run Training

In [14]:
# Fix permission issue
!chmod a+x train_haar.sh

In [15]:
!./train_haar.sh

Info file name: /content/haar-cascade-custom-training/object-detector/training.txt
Img file name: (NULL)
Vec file name: positive.vec
BG  file name: (NULL)
Num: 600
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 24
Height: 24
Max Scale: -1
Create training samples from images collection...
Done. Created 600 samples
PARAMETERS:
cascadeDirName: /content/haar-cascade-custom-training/object-detector/cascade_training/results
vecFileName: positive.vec
bgFileName: /content/haar-cascade-custom-training/object-detector/background.txt
numPos: 400
numNeg: 800
numStages: 20
precalcValBufSize[Mb] : 1024
precalcIdxBufSize[Mb] : 1024
acceptanceRatioBreakValue : -1
stageType: BOOST
featureType: HAAR
sampleWidth: 24
sampleHeight: 24
boostType: GAB
minHitRate: 0.995
maxFalseAlarmRate: 0.5
weightTrimRate: 0.95
maxDepth: 1
maxWeakCount: 100
mode: BASIC
Number of unique features given windowSize [24,24] : 16